### 1. Importación de librerías 

En las siguientes líneas de código se importan las librerías y herramientas necesarias para desarrollar el caso de uso.

In [45]:
# Librería para comando de sistema
import os
# Librerías para manejo de datos
import pandas as pd
# Para realizar la separación del conjunto de aprendizaje en entrenamiento y test.
from sklearn.model_selection import train_test_split
# Para la creación de modelo Ridge
from sklearn.linear_model import Ridge
# Para la creación de modelo Ridge
from sklearn.linear_model import Lasso
# Para búsqueda de hiperparámetros
from sklearn.model_selection import GridSearchCV
# Para la validación cruzada
from sklearn.model_selection import KFold 
# Para determinar el rendimiento del modelo con las métricas MSE, MAE y R2
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

### 2. Carga de los datos
A través de la librería **pandas** podemos realizar la carga de datos desde diferentes fuentes de información, en este caso se realizará la carga de un archivo plano csv (archivo separado por comas).

In [2]:
# Se cargan los datos. 
data=pd.read_csv('venta_inmuebles_data.csv',sep=';', encoding='latin-1')

In [3]:
# Cantidad de datos y número de variables
data.shape

(5466, 10)

In [4]:
# Mostrar los datos
data.head()

,numero_cuartos,numero_baños,area_de_ construcción_pie2,area_del lote_pie2,numero_pisos,condición,grado,superficie_sótano_pie2,año_de_construcción,precio
0,3,1,1180,5650,1,3,7,0,1955,221900
1,2,1,770,10000,1,3,6,0,1933,180000
2,4,3,1960,5000,1,5,7,910,1965,604000
3,3,2,1680,8080,1,3,8,0,1987,510000
4,3,1,1780,7470,1,3,7,730,1960,229500


### 3. Limpieza y preparación de los datos

Recuerda que un aspecto muy importante para tener en cuenta son los requerimientos de entrada de los algoritmos de aprendizaje. Cada uno de estos puede trabajar con un tipo de variable, es por esto que vamos a realizar las mismas transformaciones que se realizaron en el notebook de regresión lineal. Además, vamos a ejecutar los mismos pasos de limpieza de los datos.

In [5]:
# Es recomendable que todos los pasos de preparación se realicen sobre otro archivo.
data_t = data
# Eliminación data vacía.
data_t=data_t.dropna()
# Eliminación de registros duplicados.
data_t=data_t.drop_duplicates()

In [6]:
# Visualización de los datos limpios
data_t.head()

,numero_cuartos,numero_baños,area_de_ construcción_pie2,area_del lote_pie2,numero_pisos,condición,grado,superficie_sótano_pie2,año_de_construcción,precio
0,3,1,1180,5650,1,3,7,0,1955,221900
1,2,1,770,10000,1,3,6,0,1933,180000
2,4,3,1960,5000,1,5,7,910,1965,604000
3,3,2,1680,8080,1,3,8,0,1987,510000
4,3,1,1780,7470,1,3,7,730,1960,229500


In [7]:
# El tamaño de los datos preparados
data_t.shape

(5465, 10)

## Regresión regularizada. Caso Ridge.

### 1. Construcción del modelo

Los algoritmos supervisados implementados en scikit-learn requieren que las variables de entrada estén separadas de la variable objetivo. En este caso seguiremos con nuestra variable objetivo que es el precio. 

In [8]:
# Se selecciona la variable objetivo, en este caso "precio".
Y=data_t['precio']
# Del conjunto de datos se elimina la variable "precio".
X=data_t.drop(['precio'], axis=1)

In [9]:
# Se realiza la división entrenamiento - test. Se deja 20% de los datos para el test.
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.20, random_state = 0)

Como ya has podido conocer la regresión Ridge utiliza la norma L2 del vector de coeficientes como término de regularización. Para controlar la cantidad de regularización se utiliza el hiperparámetro alpha. Probemos con un valor igual a 2.

In [14]:
modelo_ridge = Ridge(alpha=2)
modelo_ridge

Ridge(alpha=2)

In [15]:
# Ajuste del modelo
modelo_ridge.fit(X_train,Y_train)

Ridge(alpha=2)

In [16]:
# Ahora probemos el rendimiento sobre el conjunto test.
y_pred = modelo_ridge.predict(X_test)
print("RMSE: %.2f" % mean_squared_error(Y_test, y_pred, squared=False))
print("MAE: %.2f" % mean_absolute_error(Y_test, y_pred))
print('R²: %.2f' % r2_score(Y_test, y_pred))

RMSE: 120684.72
MAE: 95104.29
R²: 0.54


#### Selección de técnica de validación para escoger el mejor alpha

##### 1-Con un conjunto de validación

In [17]:
# Dividimos el conjunto de entrenamiento en dos: una para la construcción del modelo (sería el nuevo conjunto de 
# entrenamiento) y otro para la validación, el cuál será utilizado para determinar el rendimiento del modelo con una
# combinación específica de hiperparámetros.
X_trainval, X_val, Y_trainval, Y_val = train_test_split(X_train, Y_train, test_size = 0.15, random_state = 0)

In [19]:
# Fijemos un valor de alpha en 1, construyamos el modelo y probemos sobre validación. Utilicemos MAE como métrica de
# rendimiento para hacer la selección.
modelo_ridge = Ridge(alpha=1)
modelo_ridge.fit(X_trainval,Y_trainval)
y_pred = modelo_ridge.predict(X_val)
print("MAE: %.2f" % mean_absolute_error(Y_val, y_pred))

MAE: 94544.63


In [20]:
# Fijemos un valor de alpha de 2, construyamos el modelo y probemos sobre validación.
modelo_ridge = Ridge(alpha=2)
modelo_ridge.fit(X_trainval,Y_trainval)
y_pred = modelo_ridge.predict(X_val)
print("MAE: %.2f" % mean_absolute_error(Y_val, y_pred))

MAE: 94545.31


In [21]:
# Fijemos un valor de alpha de 5, construyamos el modelo y probemos sobre validación.
modelo_ridge = Ridge(alpha=5)
modelo_ridge.fit(X_trainval,Y_trainval)
y_pred = modelo_ridge.predict(X_val)
print("MAE: %.2f" % mean_absolute_error(Y_val, y_pred))

MAE: 94547.33


El mejor alpha, según MAE, es 1. Seleccionemos este modelo como el final (por esta razón también estás técnicas se 
conocen como métodos de selección de modelos) y determinemos su rendimiento de generalización. 
Toma en cuenta que ahora debes utilizar todo el conjunto de entrenamiento.

In [23]:
modelo_ridge = Ridge(alpha=1) 
modelo_ridge.fit(X_train,Y_train)
y_pred = modelo_ridge.predict(X_test)
print("RMSE: %.2f" % mean_squared_error(Y_test, y_pred, squared=False))
print("MAE: %.2f" % mean_absolute_error(Y_test, y_pred))
print('R²: %.2f' % r2_score(Y_test, y_pred))

RMSE: 120685.40
MAE: 95102.60
R²: 0.54


Estos pasos de asignar un valor de alpha, entrenar y probar con validación pueden ser implementados en un ciclo. 

##### 2-Validación cruzada de k-particiones

In [33]:
# Fijemos el número de particiones. Utilizaremos K = 10.
particiones = KFold(n_splits=10, shuffle=True, random_state = 0)

In [34]:
# Ahora tenemos que definir el espacio de búsqueda, es decir, los valores de alpha que queremos que sean considerados. 
# Para esto se define un diccionario (o grilla) con los valores que podrá asumir el hiperparámetro alpha.
# Probemos con los siguientes valores:
param_grid = {'alpha': [1,2,3,4,5]}

In [35]:
# Definimos el modelo sin ningún valor del hiperparámetro alpha
modelo_ridge = Ridge()

In [36]:
# Ahora utilizamos GridSearch sobre el grid definido y con 10 particiones en la validación cruzada.
mejor_modelo = GridSearchCV(modelo_ridge, param_grid, cv=particiones, n_jobs=-1)
# Ajuste del modelo
mejor_modelo.fit(X_train, Y_train)

GridSearchCV(cv=KFold(n_splits=10, random_state=0, shuffle=True),
             estimator=Ridge(), n_jobs=-1,
             param_grid={'alpha': [1, 2, 3, 4, 5]})

In [37]:
# Podemos ver cual fue el resultado de la búsqueda (mejor valor de alpha)
print("Mejor parámetro: {}".format(mejor_modelo.best_params_)) 

Mejor parámetro: {'alpha': 5}


In [39]:
# También puedes indicarle a GridSearch que seleccione el mejor modelo a partir de la búsqueda con base en una métrica 
# particular. Por ejemplo, hubiésemos podido utilizar la siguiente línea de comando:
mejor_modelo = GridSearchCV(modelo_ridge, param_grid, scoring = 'neg_mean_absolute_error', n_jobs=-1)
mejor_modelo.fit(X_train, Y_train)
print("Mejor parámetro: {}".format(mejor_modelo.best_params_)) 

Mejor parámetro: {'alpha': 1}


Un aspecto importante que debes tener en cuenta es que GridSearchCV devuelve el modelo entrenado con el mejor valor del hiperparámetro alpha (resultado de la búsqueda).

In [40]:
modelo_final = mejor_modelo.best_estimator_
# Probemos ahora este modelo sobre test.
y_pred = modelo_final.predict(X_test)
print("RMSE: %.2f" % mean_squared_error(Y_test, y_pred, squared=False))
print("MAE: %.2f" % mean_absolute_error(Y_test, y_pred))
print('R²: %.2f' % r2_score(Y_test, y_pred))

RMSE: 120685.40
MAE: 95102.60
R²: 0.54


## Regresión regularizada. Caso Lasso.

### 1. Construcción del modelo

In [42]:
# Se selecciona la variable objetivo, en este caso "precio".
Y=data_t['precio']
# Del conjunto de datos se elimina la variable "precio".
X=data_t.drop(['precio'], axis=1)

In [43]:
# Se realiza la división entrenamiento - test. Se deja 20% de los datos para el test.
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.20, random_state = 0)

In [48]:
# Para acelerar la convergencia del algoritmo que utiliza Lasso para optimizar la función de costo, utilizaremos la opción
# de normalizar los datos para que todos estén en el mismo rango.
modelo_lasso = Lasso(alpha=1)
modelo_lasso

Lasso(alpha=1)

In [49]:
# Ajuste del modelo
modelo_lasso.fit(X_train,Y_train)

Lasso(alpha=1)

In [50]:
# Ahora probemos el rendimiento sobre el conjunto test.
y_pred = modelo_lasso.predict(X_test)
print("RMSE: %.2f" % mean_squared_error(Y_test, y_pred, squared=False))
print("MAE: %.2f" % mean_absolute_error(Y_test, y_pred))
print('R²: %.2f' % r2_score(Y_test, y_pred))

RMSE: 120686.13
MAE: 95101.17
R²: 0.54


#### Selección de técnica de validación para escoger el mejor alpha

##### 1-Validación cruzada de k-particiones

In [51]:
# Fijemos el número de particiones. Utilizaremos K = 10.
particiones = KFold(n_splits=10, shuffle=True, random_state = 0)

In [52]:
# Ahora tenemos que definir el espacio de búsqueda, es decir, los valores de alpha que queremos que sean considerados. 
# Para esto se define un diccionario (o grilla) con los valores que podrá asumir el hiperparámetro alpha.
# Probemos con los siguientes valores:
param_grid = {'alpha': [1, 2, 3, 4, 5]}

In [53]:
# Definimos el modelo sin ningún valor del hiperparámetro alpha
modelo_lasso = Lasso()

In [54]:
# Ahora utilizamos GridSearch sobre el grid definido y con 10 particiones en la validación cruzada.
mejor_modelo2 = GridSearchCV(modelo_lasso, param_grid, cv=particiones, n_jobs=-1)
# Ajuste del modelo
mejor_modelo2.fit(X_train, Y_train)

GridSearchCV(cv=KFold(n_splits=10, random_state=0, shuffle=True),
             estimator=Lasso(), n_jobs=-1,
             param_grid={'alpha': [1, 2, 3, 4, 5]})

In [55]:
# Podemos ver cual fue el resultado de la búsqueda (mejor valor de alpha)
print("Mejor parámetro: {}".format(mejor_modelo.best_params_))

Mejor parámetro: {'alpha': 1}


In [56]:
# También puedes indicarle a GridSearch que seleccione el mejor modelo a partir de la búsqueda con base en una métrica 
# particular. Por ejemplo, hubiésemos podido utilizar la siguiente línea de comando:
mejor_modelo2 = GridSearchCV(modelo_lasso, param_grid, scoring = 'neg_mean_absolute_error', n_jobs=-1 )
mejor_modelo2.fit(X_train, Y_train)
print("Mejor parámetro: {}".format(mejor_modelo.best_params_)) 

Mejor parámetro: {'alpha': 1}


In [57]:
modelo_final = mejor_modelo.best_estimator_
# Probemos ahora este modelo sobre test.
y_pred = modelo_final.predict(X_test)
print("RMSE: %.2f" % mean_squared_error(Y_test, y_pred, squared=False))
print("MAE: %.2f" % mean_absolute_error(Y_test, y_pred))
print('R²: %.2f' % r2_score(Y_test, y_pred))

RMSE: 120685.40
MAE: 95102.60
R²: 0.54


### 2. Importancia de variables y selección

In [58]:
# Revisar los parámetros del modelo entrenado
coeficientes = modelo_final.coef_
variables = X_train.columns
# Mostrar en una tabla los valores de los coeficientes para cada variable
pd.DataFrame({'coeficientes':coeficientes,'variables':variables})

,coeficientes,variables
0,-16514.028378,numero_cuartos
1,31665.492771,numero_baños
2,80.353658,area_de_ construcción_pie2
3,0.076465,area_del lote_pie2
4,44678.328898,numero_pisos
5,10794.855629,condición
6,91094.531022,grado
7,13.562954,superficie_sótano_pie2
8,-2581.011196,año_de_construcción
